In [46]:
import sqlite3
import pandas as pd

In [47]:
db_name = 'b3.db'
database_folder = 'backend/data'


In [100]:
def load_database(db_name):
    """Load each table from the SQLite database into its own DataFrame."""
    conn = sqlite3.connect(f'{database_folder}/{db_name}')
    
    # Define a function to get table names
    def get_table_names():
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        return [table[0] for table in tables]
    
    # Initialize a dictionary to hold DataFrames for each table
    table_dfs = {}
    
    # Get the list of table names
    table_names = get_table_names()
    
    # Load each table into its own DataFrame
    for table_name in table_names:
        query = f"SELECT * FROM {table_name};"
        table_dfs[table_name] = pd.read_sql_query(query, conn)
    
    conn.close()
    
    return table_dfs



In [101]:
table_dfs = load_database(db_name)

In [93]:
table_dfs.keys()

dict_keys(['nsd', 'company_info'])

In [94]:
df = table_dfs['company_info']
df

,company_name,ticker,pregao,listagem,cvm_code,activity,setor,subsetor,segmento,cnpj,website,ticker_codes,isin_codes,escriturador
0,1461 INVESTIMENTOS SA,UQMU,1461 INVEST,,900049,,,,,,,,,
1,3R PETROLEUM OLEO E GAS SA,RRRP,3R PETROLEUM,CIA NOVO MERCADO,25291,Exploração e produção de petróleo e gás natur...,PETROLEO GAS E BIOCOMBUSTIVEIS,PETROLEO GAS E BIOCOMBUSTIVEIS,EXPLORACAO REFINO E DISTRIBUICAO,12091809000155,,RRRP3,BRRRRPACNOR5,ITAU CORRETORA ACOES
2,3R POTIGUAR SA,PTGU,3RPOTIGUAR,,900025,,,,,,,,,
3,3Z REALTY DESENVOLVIMENTO IMOBILIARIO SA,TZRD,3ZREALTY,,908830,,NAO CLASSIFICADOS,NAO CLASSIFICADO,NAO CLASSIFICADOS,08145855000112,,,,
4,524 PARTICIPACOES SA,QVQP,524 PARTICIP,,16284,Holding - participação em outras sociedades,OUTROS,OUTROS,OUTROS,01851771000155,http://www.524participacoes.com.br/home/index....,QVQP3,BRQVQPACNOR1,ITAU CORRETORA ACOES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2736,XTRACKERS MSCI WORLD UCITS ETF,BWOR,XTRACKERWRLD,BDR DE ETF,50850,,NAO CLASSIFICADOS,NAO CLASSIFICADO,NAO CLASSIFICADOS,,,BWOR39,BRBWORBDR006,
2737,XTRACKERS MSCI WORLD VALUE UCITS ETF,BUFF,XTRACKERWRVL,BDR DE ETF,50851,,NAO CLASSIFICADOS,NAO CLASSIFICADO,NAO CLASSIFICADOS,,,BUFF39,BRBUFFBDR003,
2738,XTRACKERS RUSSELL 2000 UCITS ETF,BTWO,XTRACKER2000,BDR DE ETF,50854,,NAO CLASSIFICADOS,NAO CLASSIFICADO,NAO CLASSIFICADOS,,,BTWO39,BRBTWOBDR002,
2739,XTRACKERS SP 500 EQUAL WEIGHT UCITS ETF,BEQW,XTRACKERWEIG,BDR DE ETF,50852,,NAO CLASSIFICADOS,NAO CLASSIFICADO,NAO CLASSIFICADOS,,,BEQW39,BRBEQWBDR009,


In [102]:
df = table_dfs['nsd']
df

,nsd,company,dri,nsd_type,version,auditor,auditor_rt,protocolo,quarter,sent_date,reason
0,34,BAESAENERGETICA BARRA GRANDE SA,CARLOS ALBERTO BEZERRA DE MIRANDA,FORMULARIO CADASTRAL,1,KPMG AUDITORES INDEPENDENTES,MARCELO LIMA TONINI,019607FCA00002010010000003487,2010-01-01T00:00:00,2010-03-05T16:22:04,
1,35,PRUMO LOGISTICA SA,OTAVIO DE GARCIA LAZCANO,FORMULARIO CADASTRAL,1,KPMG AUDITORES INDEPENDENTES,MANUEL FERNANDES RODRIGUES DE SOUSA,021482FCA00002010010000003588,2010-01-01T00:00:00,2010-03-05T16:35:55,
2,36,CIA SANEAMENTO BASICO EST SAO PAULO,RUI DE BRITTO ALVARES AFFONSO,FORMULARIO CADASTRAL,1,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES,PAULO CESAR ESTEVAO NETTO,014443FCA00002010010000003680,2010-01-01T00:00:00,2010-03-05T18:41:51,
3,37,EVEN CONSTRUTORA E INCORPORADORA SA,DANY MUSZKAT,FORMULARIO CADASTRAL,1,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES,VALDIR RENATO COSCODAI,020524FCA00002010010000003782,2010-01-01T00:00:00,2010-03-09T17:00:41,
4,39,JBS SA,JEREMIAH ALPHONSUS OCALLAGHAN,FORMULARIO CADASTRAL,1,BDO TREVISAN AUDITORES INDEPENDENTES,ESTEFAN GEORGE HADDAD,020575FCA00002010010000003982,2010-01-01T00:00:00,2010-03-11T20:02:44,
...,...,...,...,...,...,...,...,...,...,...,...
49572,52222,POSITIVO TECNOLOGIA SA,LINCON LOPES FERRAZ,FORMULARIO DE REFERENCIA,5,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENTES,COSME DOS SANTOS,020362FRE20152015050005222269,2015-01-01T00:00:00,2015-11-27T15:36:18,124 126
49573,52223,DOMMO EMPREENDIMENTOS IMOBILIARIOS SA,FLAVIO NICOLAY GUIMARAES,FORMULARIO CADASTRAL,5,KPMG AUDITORES INDEPENDENTES,JOSE LUIZ DE SOUZA GURGEL,019070FCA00002015050005222369,2015-01-01T00:00:00,2015-11-27T15:36:43,ALTERACAO DO SETOR DE ATIVIDADE DA COMPANHIA C...
49574,52224,CTX PARTICIPACOES SA,FERNANDO MAGALHAES PORTELLA,FORMULARIO CADASTRAL,3,KPMG AUDITORES INDEPENDENTES,MANUEL FERNANDES RODRIGUES DE SOUSA,021652FCA00002015030005222461,2015-01-01T00:00:00,2015-11-27T16:10:04,MUDANCA DE ENDERECO DA SEDE DA COMPANHIA DELIB...
49575,52225,CTX PARTICIPACOES SA,FERNANDO MAGALHAES PORTELLA,INFORMACOES TRIMESTRAIS,2,KPMG AUDITORES INDEPENDENTES,MANUEL FERNANDES RODRIGUES DE SOUSA,021652ITR30092015020005222569,2015-09-30T00:00:00,2015-11-27T16:32:03,CORRECAO DA ESCALA DA QUANTIDADE DE ACOES DA C...
